# QAACCOUNT 账户类

QA_Account() 是quantaxis的核心类, 其作用是一个可以使用规则兼容各种市场的账户类


## 调用方式

In [2]:
import QUANTAXIS as QA
QA.QA_Account()

< QA_Account Acc_nPadcmUk market: stock_cn>

## 参数详解

In [ ]:
# strategy_name=None,
# user_cookie=None,
# portfolio_cookie=None,
# account_cookie=None,
# market_type=MARKET_TYPE.STOCK_CN,
# frequence=FREQUENCE.DAY,
# broker=BROKER_TYPE.BACKETEST,
# init_hold={},
# init_cash=1000000,
# commission_coeff=0.00025,
# tax_coeff=0.001,
# margin_level={},
# allow_t0=False,
# allow_sellopen=False,
# allow_margin=False,
# running_environment=RUNNING_ENVIRONMENT.BACKETEST

## 基于规则实例化

基于不同市场的不同规则, 我们可以实例化不同的账户类

- 允许保证金交易:  allow_marigin = True

- 允许买入后当日卖出: allow_t0 = True

- 允许卖空开仓(裸卖空): allow_sellopen= True

### 股票普通账户

In [ ]:
stock_account= QA.QA_Account(allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

###   股票融资融券账户

In [ ]:
rzrq_account = QA.QA_Account(allow_t0=False,allow_margin=True,allow_sellopen=True,running_environment=QA.MARKET_TYPE.STOCK_CN)

###  期货账户

In [ ]:
future_account = QA.QA_Account(allow_t0=True,allow_margin=True,allow_sellopen=True, running_environment=QA.MARKET_TYPE.FUTURE_CN)

### 期权账户

In [ ]:
option_account = QA.QA_Account(allow_t0=True,allow_margin=True,allow_sellopen=True, running_environment=QA.MARKET_TYPE.OPTION_CN)

###  其他市场账户


In [ ]:
xxx = QA.QA_Account(allow_t0=True,allow_margin=True,allow_sellopen=True, running_environment=QA.MARKET_TYPE.CRYPTOCURRENCY)

## 账户的初始资金/初始仓位

默认账户是无仓位, 默认现金 1000000 RMB


In [ ]:
stock_account.init_assets

In [ ]:
stock_account.init_cash

In [ ]:
stock_account.init_hold

###   在实例化的时候初始化仓位信息

使用json/dict的格式初始化  

```python
# init_hold参数
init_hold={code1:amount1,code2:amount2}
```

实例化完 会显示在 account.hold中

[注意] 在t+1的账户中, 初始化仓位依然可以当日交易

In [ ]:
stock_account= QA.QA_Account(init_hold={'000001':500}, allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

In [ ]:
stock_account.init_assets

In [ ]:
stock_account.hold

In [ ]:
stock_account.sell_available

### 在实例化的时候初始现金信息

```python
# init_cash 参数
init_cash= 200000
```


In [ ]:
stock_account= QA.QA_Account(init_cash= 200000,init_hold={'000002':500}, allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

In [ ]:
stock_account.init_assets

In [ ]:
stock_account.init_cash

In [ ]:
stock_account.cash

In [ ]:
stock_account.cash_available

### 在已经实例化的账户中修改现金/ 重置现金操作

此操作无法撤销

- 现金记录全部消除
- 账户的持仓不会消除



In [ ]:
stock_account.reset_assets(init_cash=50000)

In [ ]:
stock_account.init_assets

In [ ]:
stock_account.cash

In [ ]:
stock_account.cash_available

##  买入/卖出操作

有2种方式可以买入/卖出品种

1. send_order 接口 + Order类调用receive_deal 接口
2. receive_simpledeal 接口


其中 
- 1 是基于账户-发出订单- 订单成交回报的模式更新账户
- 2 是直接修改账户的模式, 由用户自行确认和保证成交


###  订单/更新账户的模式

在此我们只演示订单/成交, 至于order.trade怎么成交, 则是市场类/broker类的问题, 在此不去演示

#### 生成订单

In [ ]:
order=stock_account.send_order(code='000001',amount=100,time='2019-01-19',
                         amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,order_model=QA.ORDER_MODEL.CLOSE,
                         price=10,towards=QA.ORDER_DIRECTION.BUY)

In [ ]:
future_order=future_account.send_order(code='RB1905',amount=100,time='2019-01-19',
                         amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,order_model=QA.ORDER_MODEL.CLOSE,
                         price=3500,towards=QA.ORDER_DIRECTION.BUY_OPEN)

#### 订单类 QA_Order

In [ ]:
order

In [ ]:
future_order

#### 订单交易(成交在Market/Broker中判断,此处只演示成交回报)

In [ ]:
order.trade(trade_price=10.1,trade_amount=100,trade_id='example_trade1',trade_time='2019-01-19 15:00:00')

In [ ]:
future_order.trade(trade_price=3600,trade_amount=100,trade_id='example_trade2',trade_time='2019-01-19 21:00:00')

#### 账户成交表  QA_Account.history/QA_Account.history_table

In [ ]:
stock_account.history_table

In [ ]:
future_account.history_table

#### 账户现金表/ 期货账户还会冻结资金(保证金)

In [ ]:
stock_account.cash

In [ ]:
future_account.cash

In [ ]:
future_account.frozen

#### 账户可用现金

In [ ]:
stock_account.cash_available

In [ ]:
future_account.cash_available

#### 账户持仓

In [ ]:
stock_account.hold

In [ ]:
future_account.hold

#### 账户可卖余额(股票T+1 因此可卖部分不增加/ 期货账户 t+0 因此可卖部分增加)

In [ ]:
stock_account.sell_available

In [ ]:
future_account.sell_available

###    直接操作账户模式

该模式直接操作账户

In [ ]:
stock_account.receive_simpledeal(code='000004',order_id='model2',realorder_id='model2_real',trade_id='trade2',trade_amount=1000,trade_price=16,trade_time='2019-01-21',trade_towards=QA.ORDER_DIRECTION.BUY,message='model2')

In [ ]:
stock_account.history_table

In [ ]:
stock_account.cash

In [ ]:
stock_account.cash_available

In [ ]:
stock_account.hold

In [ ]:
stock_account.sell_available

##   结算

主要是股票账户的可卖股数的结算

目前期货账户不采用逐日盯市的结算方式(及无结算价/无当日盈亏)

In [ ]:
stock_account.settle()

In [ ]:
stock_account.sell_available

In [ ]:
future_account.settle()

In [ ]:
##  关于账户存储和恢复

##       存储

In [ ]:
#stock_account.save()